# Lab 8
This is a jupyter notebook for lab 8. First we will import some libraries and functions. 

Note that this doesn't work yet!

In [ ]:
from eofs.standard import Eof
from eofs.examples import example_data_path
from numpy import genfromtxt
import numpy as np
from netCDF4 import MFDataset 
from netCDF4 import Dataset 
import matplotlib
matplotlib.use('Agg')
import matplotlib.pylab as plt
from datetime import datetime
from netCDF4 import num2date
from netCDF4 import date2index
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import metpy.calc as mpcalc
from metpy.units import units
import xarray as xr
import cartopy.util as cutil
%matplotlib inline

Our next setep is to read in the netCDF files. 

In [ ]:
ds = xr.open_dataset('https://psl.noaa.gov/thredds/dodsC/Datasets/ncep.reanalysis.derived/pressure/air.mon.mean.nc')
ds2 = xr.open_dataset('http://psl.noaa.gov/psd/thredds/dodsC/Datasets/ncep.reanalysis.derived/pressure/hgt.mon.mean.nc')

Provide the pressure level and month you wish to utilize. Also provide the month as a string, so our caption is attractive.


In [ ]:
plevel = 850
month = 1
monthlabel = 'January'
tm = int(month)-1

In [ ]:
# Grab lat/lon values
lats = ds.lat.data[0:29]
lons = ds.lon.data

temp = ds.air.sel(level=plevel)[tm::12,0:29,:]
hght = ds2.hgt.sel(level=plevel)[tm::12,0:29,:]

In [ ]:
#print (np.shape(temptest))
print(np.shape(temp))
print (np.shape(lats))
print (np.shape(lons))

Calculate EOFs!

In [ ]:
coslat = np.cos(np.deg2rad(temp.coords['lat'].values)).clip(0., 1.)


print(coslat)
wgts = np.sqrt(coslat)[..., np.newaxis]

#solver for EOFs/PCs: initially set for 850 hPa T
solver = Eof(temp.values, weights=wgts)
#EOF calculation:leading 2 EOFs
eof1 = solver.eofsAsCorrelation(neofs=3)
#eof1 = solver.eofsAsCovariance(neofs=2)
pc1 = solver.pcs(npcs=3, pcscaling=1)
varianceexplained = solver.varianceFraction(neigs=10)


Map time!

In [ ]:
# Set up the projection that will be used for plotting
levels = np.arange(-1,1.05,0.05)

mapcrs = ccrs.Robinson(central_longitude=180)

# Set up the projection of the data; if lat/lon then PlateCarree is what you want
datacrs = ccrs.PlateCarree()

# Start the figure and create plot axes with proper projection
fig = plt.figure(1, figsize=(14, 12))
ax = plt.subplot(111, projection=mapcrs)

#not needed here, but used for sub-setting domain to plot
#ax.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())

# Add geopolitical boundaries for map reference
ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax.add_feature(cfeature.STATES.with_scale('50m'))
gl = ax.gridlines(color='gray',linewidth=2, alpha = 0.5, linestyle='--')

# labels gridlines for PlateCarree ccrs option (doesn't work with Robinson, etc.)
#gl.xlabels_top = False
#gl.ylabels_right = False

#color-filled contour time!
#plotting temperature and including color bar

cf = ax.contourf(lons, lats, eof1[0,:,:], levels, cmap=plt.cm.RdYlBu_r, transform=datacrs)
plt.colorbar(cf, orientation='horizontal', pad=0.05, aspect=50)
cs = ax.contour(lons, lats, eof1[0,:,:], levels, colors='black', transform=datacrs)
plt.clabel(cs, fmt='%1.1f')

# Plot 500-hPa wind barbs in knots, regrid to reduce number of barbs
#ax.barbs(lons[::4], lats[::4], uwnd.to('kt')[ty,::4,::4].m, 
#         vwnd.to('kt')[ty,::4,::4].m, pivot='middle',
#         color='black', transform=datacrs, length=5, linewidth=0.4)

# Make some nice titles for the plot (one right, one left)
plt.title('850 hPa T EOF 1 ' + str(monthlabel))

# Reduce excessive white space around image and show
plt.tight_layout()
plt.savefig('850hPaTeof1' + str(monthlabel), dpi=150)
plt.show()

In [ ]:
#Associated PC time series:
years = list(range(1948,1948+len(pc1[:,0])))
plt.plot(years,pc1[:,0])
plt.axis([1948,2016,-3,3])
plt.title(str(monthlabel) + ' ' +'PC1')
plt.show()

Histogram!

In [ ]:
plt.hist(pc1[:,0])
plt.axis([-3,3.1,0,20])
plt.title(str(monthlabel) + ' ' +'PC1')
plt.show()

In [ ]:
print ('pc np shape')
print (np.shape(pc1))
print (pc1[:,0])
print ('varianceexplained')
print (varianceexplained)
print (np.shape(eof1))

Plot the 2nd EOF!

In [ ]:
plt.figure(2,figsize=(14, 12))

# Set up the projection that will be used for plotting
levels = np.arange(-1,1.05,0.05)
mapcrs = ccrs.Robinson(central_longitude=180)
# Set up the projection of the data; if lat/lon then PlateCarree is what you want
datacrs = ccrs.PlateCarree()

# Start the figure and create plot axes with proper projection
fig = plt.figure(2, figsize=(14, 12))
ax = plt.subplot(111, projection=mapcrs)

#not needed here, but used for sub-setting domain to plot
#ax.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())

# Add geopolitical boundaries for map reference
ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax.add_feature(cfeature.STATES.with_scale('50m'))
gl = ax.gridlines(color='gray',linewidth=2, alpha = 0.5, linestyle='--')

# labels gridlines for PlateCarree ccrs option (doesn't work with Robinson, etc.)
#gl.xlabels_top = False
#gl.ylabels_right = False

#color-filled contour time!
#plotting temperature and including color bar

cf = ax.contourf(lons, lats, eof1[1,:,:], levels, cmap=plt.cm.RdYlBu_r, transform=datacrs)
plt.colorbar(cf, orientation='horizontal', pad=0.05, aspect=50)
cs = ax.contour(lons, lats, eof1[1,:,:], levels, colors='black', transform=datacrs)
plt.clabel(cs, fmt='%1.1f')

# Plot 500-hPa wind barbs in knots, regrid to reduce number of barbs
#ax.barbs(lons[::4], lats[::4], uwnd.to('kt')[ty,::4,::4].m, 
#         vwnd.to('kt')[ty,::4,::4].m, pivot='middle',
#         color='black', transform=datacrs, length=5, linewidth=0.4)

# Make some nice titles for the plot (one right, one left)
plt.title('850 hPa T EOF 2 ' + str(monthlabel))

# Reduce excessive white space around image and show
plt.tight_layout()
plt.savefig('850hPaTeof2' + str(monthlabel), dpi=150)
plt.show()

In [ ]:
#Associated PC time series:
years = list(range(1948,1948+len(pc1[:,1])))
plt.plot(years,pc1[:,1])
plt.axis([1948,2016,-3,3])
plt.title(str(monthlabel) + ' ' +'PC2')
plt.show()

In [ ]:
plt.hist(pc1[:,1])
plt.axis([-3,3.1,0,20])
plt.title(str(monthlabel) + ' ' +'PC2')
plt.show()

Write the PC time series to a file

In [ ]:
outfile = open('850hPaTPC' + str(monthlabel)+'.csv','w')
outfile.writelines("Year   PC1      PC2 \n")
#outfile.writelines(str(pc1[:,0])+', '+str(pc1[:,1])+'\n')
for i in range(0,len(pc1)):
   outfile.writelines(str(years[i])+', '+str(pc1[i,0])+', '+str(pc1[i,1])+'\n')
outfile.close()


Read in NAO data, so we can see relationship with temperature PCs!

In [ ]:
infile = open('../data/mei.txt', 'r')
mei = infile.readlines()
print (np.shape(mei))

for i in range(len(mei)):
  mei[i] = mei[i].split()

mei = np.array(mei).astype('float')
print (np.shape(mei))

Scatterplot of the leading 850 hPa temperature PC with the NAO  

In [ ]:
plt.plot(pc1[2:69,0],mei[2:69,1], 'bo')
#plt.axis([-2, 2, -2, 2])
plt.title('850 hPa Temperature PC1 and January MEI')
plt.xlabel(' Temperature PC1')
plt.ylabel('MEI')
plt.show()

In [ ]:
print(np.corrcoef(pc1[2:69,0],mei[2:69,1])[1,0])